In [6]:
using SparseArrays
using LinearAlgebra

In [7]:
Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];

In [8]:
X = [0 1;1 0];

In [9]:
"""

Following function takes a 2x2 matrix (Gate) and qubit position (Qubit) and
returns the resultant matrix.

For example, the matrix for the gate U acting on the 3-rd qubit for N=5
qubit system is given by   I (x) I (x) U (x) I (x) I; where (x) is the
tensor product.

"""

function Matrix_Gate(Gate, Qubit)
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:N
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:N
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end

Matrix_Gate (generic function with 1 method)

In [10]:
Identity(dimension) = 1* Matrix(I, dimension, dimension);
#Identity(3)

## Single controlled unitary gate

In [106]:
"""

The following function returns a controlled U gate matrix.

Input  : c (integer), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubit c and target qubit t.

"""

function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;               

## Multi controlled unitary gate

In [12]:
"""

The following returns a multicontrolled U gate matrix.

Input  : c (list), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubits c and target qubit t.

"""

function MCU(c,t,U)
    
    p0 = fill("I", N)
    p1 = fill("I", N)

    
    if typeof(c) == Int64
        p0[c] = "PI_1"
        p1[t] = "PI_1"
        
    else
        for i in c
            p0[i] = "PI_1"
            p1[i] = "PI_1"
        end
    end
    
    p0[t] = "I"
    p1[t] = "U"

    
    I = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])
    X = sparse([0 1;1 0])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2
     
    Matrices = Dict("I" => I,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:N
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:N
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
             
    # The identity in the following line needs to be replaced.
    return Identity(2^N) - PI_0_matrix + PI_1_matrix     
end;             

## Toffoli test

In [13]:
N = 3
TOF = CU(Rx(pi/2),2,3)*CU(Rx(pi),1,2)*CU(Rx(pi/2),1,3)*CU(Rx(-pi/2),2,3)*CU(Rx(-pi),1,2);
map(x->round.(x), TOF) 

8×8 SparseMatrixCSC{ComplexF64, Int64} with 8 stored entries:
 1.0+0.0im      ⋅          ⋅      …      ⋅          ⋅          ⋅    
     ⋅      1.0+0.0im      ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅      1.0+0.0im         ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅      …  1.0+0.0im      ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅      0.0-1.0im
     ⋅          ⋅          ⋅             ⋅      0.0-1.0im      ⋅    

## L = 8

In [14]:
N = 8
C1 = [
    CU(Rx(pi/2),7,8),

    CU(Rx(pi/2),6,7),
    CU(Rx(pi/2^2),6,8),
    
    CU(Rx(pi/2),5,6),
    CU(Rx(pi/2^2),5,7),
    CU(Rx(pi/2^3),5,8),
    
    CU(Rx(pi/2),4,5),
    CU(Rx(pi/2^2),4,6),
    CU(Rx(pi/2^3),4,7),
    CU(Rx(pi/2^4),4,8),
    
    CU(Rx(pi/2),3,4),
    CU(Rx(pi/2^2),3,5),
    CU(Rx(pi/2^3),3,6),
    CU(Rx(pi/2^4),3,7),
    CU(Rx(pi/2^5),3,8),

    CU(Rx(pi/2),2,3),
    CU(Rx(pi/2^2),2,4),
    CU(Rx(pi/2^3),2,5),
    CU(Rx(pi/2^4),2,6),
    CU(Rx(pi/2^5),2,7),
    CU(Rx(pi/2^6),2,8)
];

C2 = [
    CU(Rx(pi),1,2),
    CU(Rx(pi/2),1,3),
    CU(Rx(pi/2^2),1,4),
    CU(Rx(pi/2^3),1,5),
    CU(Rx(pi/2^4),1,6),
    CU(Rx(pi/2^5),1,7),
    CU(Rx(pi/2^6),1,8)
];

C3 = [
    CU(Rx(-pi/2),2,3),
    CU(Rx(-pi/2^2),2,4),
    CU(Rx(-pi/2^3),2,5),
    CU(Rx(-pi/2^4),2,6),
    CU(Rx(-pi/2^5),2,7),
    CU(Rx(-pi/2^6),2,8),
    
    CU(Rx(-pi/2),3,4),
    CU(Rx(-pi/2^2),3,5),
    CU(Rx(-pi/2^3),3,6),
    CU(Rx(-pi/2^4),3,7),
    CU(Rx(-pi/2^5),3,8),
    
    CU(Rx(-pi/2),4,5),
    CU(Rx(-pi/2^2),4,6),
    CU(Rx(-pi/2^3),4,7),
    CU(Rx(-pi/2^4),4,8),
    
    CU(Rx(-pi/2),5,6),
    CU(Rx(-pi/2^2),5,7),
    CU(Rx(-pi/2^3),5,8),
    
    CU(Rx(-pi/2),6,7),
    CU(Rx(-pi/2^2),6,8),
    
    CU(Rx(-pi/2),7,8)
];

C4 = [
    CU(Rx(pi/2),6,7),
    
    CU(Rx(pi/2),5,6),
    CU(Rx(pi/2^2),5,7),
    
    CU(Rx(pi/2),4,5),
    CU(Rx(pi/2^2),4,6),
    CU(Rx(pi/2^3),4,7),
    
    CU(Rx(pi/2),3,4),
    CU(Rx(pi/2^2),3,5),
    CU(Rx(pi/2^3),3,6),
    CU(Rx(pi/2^4),3,7),
    
    CU(Rx(pi/2),2,3),
    CU(Rx(pi/2^2),2,4),
    CU(Rx(pi/2^3),2,5),
    CU(Rx(pi/2^4),2,6),
    CU(Rx(pi/2^5),2,7),
];

C5 = [
    CU(Rx(-pi),1,2),
    CU(Rx(-pi/2),1,3),
    CU(Rx(-pi/2^2),1,4),
    CU(Rx(-pi/2^3),1,5),
    CU(Rx(-pi/2^4),1,6),
    CU(Rx(-pi/2^5),1,7)
];

C6 = [
    CU(Rx(-pi/2),2,3),
    CU(Rx(-pi/2^2),2,4),
    CU(Rx(-pi/2^3),2,5),
    CU(Rx(-pi/2^4),2,6),
    CU(Rx(-pi/2^5),2,7),
    
    CU(Rx(-pi/2),3,4),
    CU(Rx(-pi/2^2),3,5),
    CU(Rx(-pi/2^3),3,6),
    CU(Rx(-pi/2^4),3,7),
    
    CU(Rx(-pi/2),4,5),
    CU(Rx(-pi/2^2),4,6),
    CU(Rx(-pi/2^3),4,7),
    
    CU(Rx(-pi/2),5,6),
    CU(Rx(-pi/2^2),5,7),
    
    CU(Rx(-pi/2),6,7)
];

In [15]:
C1[1]*C1[2]*C1[3]

256×256 SparseMatrixCSC{ComplexF64, Int64} with 704 stored entries:
⠱⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠛⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠱⢆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠑⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠛⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠱⢆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠱⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠱⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⢆⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⣤⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⢄⡀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⢆⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦

In [97]:
# Initializing the MCX matrix.
N = 8
MCX = Identity(2^N);

for i in C1
    MCX = i*MCX#MCX*i
end

for i in C2
    MCX = i*MCX#MCX*i
end

for i in C3
    MCX = i*MCX#MCX*i
end

for i in C4
    MCX = i*MCX#MCX*i
end

for i in C5
    MCX = i*MCX#MCX*i
end

for i in C6
    MCX = i*MCX#MCX*i
end
#MCX=MCX/MCX[1,1]
#MCX
#map(x->round.(x), MCX) 

# Circuit for general L

In [136]:
L = 5

C_1 = [];
C_2 = [];
C_3 = [];
C_4 = [];
C_5 = [];
C_6 = [];
MCX = Identity(2^L);

# C_1.
for i = 1:L-2
    for j = 1:i
        push!(C_1,[j,L-i,L-i+j])
        MCX = CU(Rx(pi/2^j), L-i, L-i+j)*MCX
    end
end

# C_2.
for i = 2:L
    push!(C_2,[i-2,1,i])
    MCX = CU(Rx(pi/2^(i-2)), 1, i)*MCX
end

# C3 = - C1.
for i = L-2:-1:1
    for j = i:-1:1
        MCX = CU(Rx(-pi/2^j), L-i, L-i+j)*MCX
    end
end

# C_4.
for i = 1:L-3
    for j = 1:i
        push!(C_4,[j,L-i-1,L-i+j-1])
        MCX = CU(Rx(pi/2^j), L-i-1, L-i-1+j)*MCX
    end    
end

# C_5.
for i = 2:L-1
    push!(C_5,[i-2,1,i])
    MCX = CU(Rx(-pi/2^(i-2)), 1, i)*MCX
end

# C6 = - C4.
for i = L-3:-1:1
    for j = i:-1:1
        MCX = CU(Rx(-pi/2^j), L-i-1, L-i-1+j)*MCX
    end    
end

In [133]:
for i = L-2:-1:1
    for j = i:-1:1
        print([j,L-i,L-i+j])
    end
end    

[3, 2, 5][2, 2, 4][1, 2, 3][2, 3, 5][1, 3, 4][1, 4, 5]

In [107]:
# Creating an empty matrix to begin with.
MCX = Identity(2^L);

for i in C_1
    MCX = CU(Rx(pi/2^i[1]), i[2], i[3])*MCX
end

for i in C_2
    MCX = CU(Rx(pi/2^i[1]), i[2], i[3])*MCX
end

for i in reverse(C_1) # C_3 = - C_1.
    MCX = CU(Rx(-pi/2^i[1]), i[2], i[3])*MCX    
end

for i in C_4
    MCX = CU(Rx(pi/2^i[1]), i[2], i[3])*MCX
end

for i in C_5
    MCX = CU(Rx(-pi/2^i[1]), i[2], i[3])*MCX
end

for i in reverse(C_4) # C_6 = - C_4.
    MCX = CU(Rx(-pi/2^i[1]), i[2], i[3])*MCX
end    

In [137]:
MCX[2^L,2^L-1],MCX[2^L-1,2^L]

(0.0 - 0.9999999999999998im, 0.0 - 0.9999999999999998im)